<a href="https://colab.research.google.com/github/9-coding/Kaggle/blob/main/ISIC_2024/image_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Requirements

In [ ]:
import time
from tqdm import tqdm
import os
import h5py
from PIL import Image
from io import BytesIO

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda import amp

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

## Configuration and Set Seed

In [ ]:
CONFIG = {
    "seed": 42,
    "img_size": 256,
    "batch_size": 1024,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(CONFIG['seed'])

In [ ]:
ROOT_DIR = "/kaggle/input/isic-2024-challenge"

TEST_HDF  = f'{ROOT_DIR}/test-image.hdf5'
TEST_CSV  = f'{ROOT_DIR}/test-metadata.csv'
IMAGE_HDF = f'{ROOT_DIR}/train-image.hdf5'
TARGET_CSV = f'{ROOT_DIR}/train-metadata.csv'
SAMPLE    = f'{ROOT_DIR}/sample_submission.csv'

## Data Configuration

In [ ]:
df = pd.read_csv(TARGET_CSV)
print(len(df))
df.head()

## Dataset and DataLoader

In [ ]:
class ISIC(Dataset):
    def __init__(self, file_hdf, df, transforms):
        self.fp_hdf = h5py.File(file_hdf, mode="r")
        self.df = df
        self.isic_ids = df['isic_id'].values
        self.targets = df['target'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.isic_ids)

    def __getitem__(self, index):
        isic_id = self.isic_ids[index]
        img = Image.open(BytesIO(self.fp_hdf[isic_id][()]))
        target = self.targets[index]

        if self.transforms:
            img = self.transforms(img)

        return {
            'image': img,
            'target': target,
        }

In [ ]:
transforms_data = transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor()])
dataset = ISIC(IMAGE_HDF, df, transforms=transforms_data)
dataset_size = len(dataset)

train_size = int(dataset_size * 0.8)                     # 80%
validation_size = int(dataset_size * 0.1)                # 10%
test_size = dataset_size - train_size - validation_size  # 10%

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(val_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, pin_memory=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, pin_memory=True, drop_last=True)

In [ ]:
### GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

In [ ]:
EPOCH = 30
lr = 0.1
model = models.resnet18(pretrained=True)

### Transfer Learning ###
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 50)
model.to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=lr)
print("Created a learning model and optimizer")

In [ ]:
### Train/Evaluation ###
def train(model, train_loader, optimizer, epoch):
  model.train()
  train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch}", leave=False)
  for i, batch in enumerate(train_loader_tqdm):
    image, target = batch['image'].to(DEVICE), batch['target'].to(DEVICE)

    output = model(image)
    optimizer.zero_grad()
    train_loss = F.cross_entropy(output, target).to(DEVICE)

    train_loss.backward()
    optimizer.step()

  return train_loss

In [ ]:
def evaluate(model, val_loader):
  model.eval()
  eval_loss = 0
  correct = 0
  with torch.no_grad():
    for i, batch in tqdm(enumerate(val_loader)):
      image, target = batch['image'].to(DEVICE), batch['target'].to(DEVICE)
      output = model(image)
      eval_loss += F.cross_entropy (output, target, reduction='sum').item()
      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as (pred)).sum().item()
    eval_loss /= len(val_loader.dataset)
    eval_accuracy = 100 * correct / len(val_loader.dataset)
    return eval_loss, eval_accuracy

In [ ]:
### Main ###
start = time.time()
best = 0
for epoch in range(EPOCH):
  train_loss = train(model, train_loader, optimizer, epoch)
  val_loss, val_accuracy = evaluate(model, val_loader)

  # Save best model
  if val_accuracy > best:
    best = val_accuracy
    torch.save(model.state_dict(), "./best_model.pth")
  print(f'[{epoch}] Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}%')

# Test result
test_loss, test_accuracy = evaluate(model, test_loader)
wandb.log({'test_accuracy': test_accuracy})
print(f'[FINAL] Test Loss {test_loss:.4f}, Accuracy: {test_accuracy:.4f}%')
end = time.time()
elasped_time = end - start

print("Best Accuracy: ", best)
print(f"Elasped Time: {int(elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")
print(f"time: {int (elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")